In [2]:
# !pip install imapclient pdfplumber
# !pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client
# import imaplib
# import email
# import os
# # import streamlit as st

# # import pdfplumber
# from googleapiclient.discovery import build
# from google.oauth2.credentials import Credentials
# from google_auth_oauthlib.flow import InstalledAppFlow
# import os

# # Define OAuth scopes
# SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]

# # Load credentials
# def get_gmail_service():
#     creds = None
#     token_file = "token.json"
#     creds_file = "creds.json"  # Replace with the path to your downloaded JSON

#     if os.path.exists(token_file):
#         creds = Credentials.from_authorized_user_file(token_file, SCOPES)

#     # If no valid credentials, login and save token
#     if not creds or not creds.valid:
#         flow = InstalledAppFlow.from_client_secrets_file(creds_file, SCOPES)
#         creds = flow.run_local_server(port=0)
#         with open(token_file, "w") as token:
#             token.write(creds.to_json())

#     return build("gmail", "v1", credentials=creds)

# # Fetch unread emails
# def get_unread_emails():
#     service = get_gmail_service()
#     results = service.users().messages().list(userId="me", labelIds=["UNREAD"]).execute()
#     messages = results.get("messages", [])

#     # for message in messages:
#     #     msg_id = message["id"]
#     #     msg = service.users().messages().get(userId="me", id=msg_id).execute()
#     #     print("Subject: {msg['snippet']}")

#     for message in messages:
#         msg_id = message["id"]
#         msg = service.users().messages().get(userId="me", id=msg_id, format="metadata").execute()
        
#         headers = msg["payload"]["headers"]
#         subject = next((header["value"] for header in headers if header["name"] == "Subject"), "No Subject")
        
#         print(f"Subject: {subject}")


# get_unread_emails()

import os
import base64
import pdfplumber
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow

# Define OAuth scopes
SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]

# Save attachments here
SAVE_DIR = "attachments"
os.makedirs(SAVE_DIR, exist_ok=True)

# Authenticate Gmail API
def get_gmail_service():
    creds = None
    token_file = "token.json"
    creds_file = "credentials.json"  # Replace with your credentials file

    if os.path.exists(token_file):
        creds = Credentials.from_authorized_user_file(token_file, SCOPES)

    if not creds or not creds.valid:
        flow = InstalledAppFlow.from_client_secrets_file(creds_file, SCOPES)
        creds = flow.run_local_server(port=0)
        with open(token_file, "w") as token:
            token.write(creds.to_json())

    return build("gmail", "v1", credentials=creds)

# Fetch unread emails
def get_unread_emails():
    service = get_gmail_service()
    results = service.users().messages().list(userId="me", labelIds=["UNREAD"]).execute()
    messages = results.get("messages", [])

    for message in messages:
        msg_id = message["id"]
        msg = service.users().messages().get(userId="me", id=msg_id).execute()
        # print(f"Processing Email: {msg['snippet'][:50]}...")  # Print snippet

        # Extract attachments
        extract_attachments(service, msg)

# Extract attachments from email
def extract_attachments(service, msg):
    msg_id = msg["id"]
    for part in msg["payload"].get("parts", []):
        if part["filename"] and "application/pdf" in part["mimeType"]:
            attachment_id = part["body"]["attachmentId"]
            attachment = service.users().messages().attachments().get(userId="me", messageId=msg_id, id=attachment_id).execute()
            
            file_data = base64.urlsafe_b64decode(attachment["data"])
            file_path = os.path.join(SAVE_DIR, part["filename"])

            with open(file_path, "wb") as f:
                f.write(file_data)

            print(f"Saved PDF: {file_path}")
            extract_text_from_pdf(file_path)

# Extract text from PDF
def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = "\n".join(page.extract_text() for page in pdf.pages if page.extract_text())
        print(f"Extracted Text from {pdf_path}:\n{text[:10000]}...\n")  # Print first 500 characters

get_unread_emails()



Saved PDF: attachments/AFGRI INV 9001811822 - UG.pdf
Extracted Text from attachments/AFGRI INV 9001811822 - UG.pdf:
COPY TAX CREDIT NOTE AFGRI Grain Silo Company (Pty) Ltd
Reg. Number: 2016/440533/07
VAT Reg No: 4130283874
Postal Address: PO Box 11054
Highveld Ext 73, Centurion
Physical Address: 12 Byls Bridge Boulevard
Centurion, 0046
Invoice No: 9001811822
Reference No: HT-2011-01013504
Document Date: 03 May 2024
UNIGRAIN (PTY) LTD
Po Box 488 Payment Details
Randfontein Bank name: ABSA
Account Name: AFGRI Grain Silo Company (Pty) Ltd
1760
Account No: 04095120081
South Africa Branch Code: 620503
VAT Reg No: 4670140054
Customer Account Details
Account No: 343158
Account Name: UNIGRAIN (PTY) LTD
Payment Terms: 30 Days after Statement
BP Main Account: 912685
Folio No: 2811894
E-Mail Address: wilne.spangenberg@unigrain.co
INVOICE DETAILS
No. Description Qty Unit Price Total (Excl.) VAT Total (Incl.)
1 MAIZE YELLOW SEASON TARIFF 1,018.895 143.23 145,936.33 21,890.45 167,826.78
SEISOENSTARI

In [269]:

import pdfplumber
import re

# Function to extract floats from text
def extract_floats(text):
    # Regular expression to capture floats
    float_pattern = r'\b\d+\.\d+\b'
    return re.findall(float_pattern, text)
dd = [
'118268',
'158505',
'2838416',
'2864370',
'2867704',
'2816302',
'2811894',
'2838429',
'148911',
'2838416',
'2816302',
'2838429',
'2867704',
'2864370',
'148911',
'158506',
'158506',
'9405257',
'9405257',
'160238',
'160238',
'165312',
'165312',
'B257201-G301',
'B114249-G301',
'B114249-G301',
'NEX021',
'UNI043',
'56786',
'7362400',
'23559',
'X200914NEX',
'V000068UNI',
'ZD01186UNI',
'X960807UNI',
'ZL00144GRA',
'415520',
'852213',
'85391',
'805247',
'104140',
'750336',
'802836',
'802962',
'GC0049',
'GC0058',
'GC0020',
'GC0001',
'9052',
'9072',
'9755',
'9101',
'9076',
'9751',
'534511',
'531221',
'534117',
'534117',
'418635',
'424782',
'401664',
'530268',
'401664',
'530268',
'22229',
'00613',
'21704',
'820449',
'821982',
'N0560162',
'N2880370',
'N3064904',
'N3104974']

cc = [
'UNIGRAIN',
'UNIGRAIN COMMODITIES',
'UNIGRAIN',
'UNIGRAIN',
'UNIGRAIN',
'UNIGRAIN',
'UNIGRAIN',
'UNIGRAIN',
'UNIGRAIN COMMODITIES',
'UNIGRAIN',
'UNIGRAIN',
'UNIGRAIN',
'UNIGRAIN',
'UNIGRAIN',
'UNIGRAIN',
'UNIGRAIN COMMODITIES',
'AGI',
'AGI',
'NEXGRO BEMARKING',
'NEXGRO BEMARKING',
'NEXGRO BEMARKING',
'NEXGRO BEMARKING',
'AGI',
'AGI',
'NEXGRO BEMARKING',
'UNIGRAIN COMMODITIES',
'UNIGRAIN',
'NEXGRO BEMARKING',
'UNIGRAIN',
'UNIGRAIN',
'NEXGRO BEMARKING',
'UNIGRAIN',
'NEXGRO BEMARKING',
'UNIGRAIN',
'UNIGRAIN',
'UNIGRAIN',
'UNIGRAIN',
'NEXGRO BEMARKING',
'UNIGRAIN',
'UNIGRAIN',
'NEXGRO BEMARKING',
'UNIGRAIN',
'ABSA / UNIGRAIN',
'AGI',
'AGI',
'AGI',
'NEXGRO BEMARKING',
'UNIGRAIN',
'UNIGRAIN COMMODITIES',
'ABSA / AGI',
'ABSA / UNIGRAIN',
'AGI',
'NEXGRO BEMARKING',
'UNIGRAIN',
'UNIGRAIN COMMODITIES',
'ABSA / AGI',
'ABSA / UNIGRAIN',
'AGI',
'AGI',
'AGI',
'NEXGRO BEMARKING',
'UNIGRAIN',
'UNIGRAIN',
'UNIGRAIN',
'UNIGRAIN',
'NEXGRO BEMARKING',
'UNIGRAIN',
'UNIGRAIN COMMODITIES',
'UNIGRAIN',
'NEXGRO BEMARKING',
'UNIGRAIN']

jj = [
'Afrikaskop silo',
'Amersfoort silo',
'Argent silo',
'Arnot silo',
'Bakenlaagte Bunker',
'Balfour Silo',
'Bapsfontein Bunker',
'Battery silo',
'Beestekraal silo',
'Bergville silo',
'Bethal silo',
'Bethlehem silo',
'Bloedrivier silo',
'Bloekomspruit silo',
'Brakfontein Bunker',
'Brits silo',
'Bronkhorstspruit silo',
'Carolina silo',
'Chelmsforddam Bunker',
'Dannhauser silo',
'Davel silo',
'Delmas Bunker',
'Dennegeur Bunker',
'Devon silo',
'Driefontein silo',
'Dryden silo',
'Dundee silo',
'Eenboom Bunker',
'Eensgezindt Bunker',
'Eeram silo',
'Eloff silo',
'Endicott silo',
'Ermelo silo',
'Estancia silo',
'Glenroy silo',
'Goeiehoek silo',
'Greylingstad silo',
'Grootvlei silo',
'Harrismith silo',
'Harvard silo',
'Hawerklip silo',
'Hendriksvallei Bunker',
'Holmdene silo',
'Kaalfontein silo',
'Kaallaagte silo',
'Kendal silo',
'Kinross Bunker',
'Kinross silo',
'Klipfontein Bunker',
'Kliprivier bunker',
'Kortlaagte Bunker',
'Kransfontein silo',
'Leeuspruit silo',
'Leslie silo',
'Libertas silo',
'Lothair silo',
'Lydenburg silo',
'Maizefield silo',
'Marble Hall silo',
'MARQUARD silo',
'Meets silo',
'Meyerton Bunker',
'Middelburg silo',
'Mispah silo',
'Monte Video silo',
'Morgenzon silo',
'NEXSTOR HOLFONTEIN DEPOT',
'Nigel silo',
'Northam silo',
'Ogies silo',
'Overvaal silo',
'PALMIETF BKR (BALFOUR)',
'Pan silo',
'Paulpietersburg silo',
'Pietermaritzburg silo',
'Platrand silo',
'Pretoria West silo',
'Senekal silo',
'Slabberts silo',
'Standerton silo',
'Stoffberg silo',
'Trichardt silo',
'Vaaldrift Bunker',
'Val silo',
'Vlakfontein Bunker',
'Vogelvallei Bunker',
'Vryheid silo',
'Winterton Bunker',
'Winterton silo',
'Wonderfontein silo',
'Bloekomhoek Depot',
'Bultfontein Depot',
'Goudkop Depot',
'Hassiesdrift Depot',
'Nutopia Depot',
'Pampoenkraal Depot',
'Vlaklaagte Depot',
'Waterval Depot',
'Winterhoek Depot',
'Wonderfontein Depot',
'Agri-Land Silo',
'Banabatau Modderfontein',
'Bellary Silo (KB)',
'Buffalo Silo',
'Bushel Silo (KB)',
'Die Bult',
'Doornkorp Silo (KB)',
'Graanland Silo (KB)',
'Grain Handlers (KB)',
'Hanover Silo',
'Ingogo Silo Newcastle (KB)',
'JJ Potgieter Storage',
'Nexgro Holfontein Silo',
'NEXSTOR HOLFONTEIN DEPOT',
'Rossgro Silo (KB)',
'Vlakvlei Grain Handling and Storage',
'Waagkraal Depot',
'Barkley West',
'Douglas',
'Modderrivier',
'Prieska',
'Stoffelshoek Bunker',
'Trans Oranje',
'Ceres',
'Darling',
'Doornkuil Bunker',
'Eendekuil',
'Klipheuwel',
'Malmesbury',
'Piketberg',
'Pools',
'Porterville',
'Riebeek Wes',
'Ruststasie',
'Baberspan',
'Barberspan',
'Biesiesvlei',
'Blaaubank',
'Bodenstein',
'Boons',
'Boschpoort',
'Bossies',
'Buhrmannsdrif',
'Coligny',
'Delareyville',
'Derby',
'Excelsior',
'Gerdau',
'Geysdorp',
'Grootpan',
'Halfpad',
'Hibernia',
'Kameel',
'Kleinharts',
'Koster',
'Lichtenburg',
'LOTTIES HALTE',
'Lottieshalte',
'Lusthof',
'Madibogo',
'Mareetsane',
'Migdol',
'Nooitgedacht',
'Oppaslaagte',
'Ottosdal',
'Rostrataville',
'Sannieshof',
'Swartruggens',
'Syferbult',
'Taaiboschpan',
'Vermaas',
'Bergrivier',
'Bredasdorp',
'Caledon',
'Jongensklip Silo',
'Klipdale',
'KRIGE',
'Lemoenskop (Caledon Bunker)',
'Moorreesburg',
'Moravia',
'Napier',
'Ouplaas Depot',
'Protem',
'Rietpoel',
'Clocolan',
'Ficksburg',
'Fouriesburg',
'Marseilles',
'Modderpoort',
'Morgenzon',
'Oranjerivier',
'Prieska',
'Thaba Nchu',
'Tweespruit',
'Westminster'
'Aureus',
'Albertinia',
'Ashton',
'HEIDELBERG',
'Karringmelksrivier',
'Protem',
'Riversdale',
'Swellendam',
'Broodsnyers',
'De Vale',
'Delpa',
'Groot Saxony',
'Helpmekaar',
'Kleinhoek',
'Mispah',
'Sarbyn',
'Schoongesicht',
'Theronia',
'Vyfsusters',
'Allanridge',
'Amalia',
'Arlington',
'Attie',
'Bamboesspruit',
'Bloemfontein',
'Bloemhof',
'Bothaville Silo nr 2',
'Brandfort',
'Buckingham Silo',
'Bultfontein',
'Christiana Silo',
'De Brug Silo',
'Enselspruit Silo',
'Geneva',
'Gottenburg',
'Groenebloem Silo',
'Hallatshope',
'Hartbeesfontein',
'Hartswater',
'Heilbron Silo',
'Hennenman',
'Hertzogville',
'Heuningspruit',
'Hoogte',
'Hoopstad Silo',
'Jan Kempdorp',
'Kameel',
'Kingswood',
'Koppies Silo',
'Kroonstad',
'Leeudoringstad Silo',
'Losdoorns',
'Magogong Silo',
'Makokskraal Silo',
'Makwassie Silo',
'Melliodora',
'Middelvlei Silo',
'Migdol',
'Mirage',
'Mooigelee',
'Oberholzer Silo',
'Odendaalsrus',
'Petrusburg',
'Poppieland Bunker',
'Potchefstroom Silo',
'Protespan',
'Raathsvlei Silo',
'Regina',
'Rooiwal',
'Schoonspruit',
'Schuttesdraai Silo',
'Schweizer Reneke Silo',
'Steynsrus',
'Strydpoort Silo',
'Theunissen Silo',
'Tierfontein Silo',
'Van Tonder Silo',
'Ventersdorp Silo',
'Vierfontein',
'Viljoenskroon',
'Vredefort',
'Vryburg',
'Weiveld',
'Welgelee',
'Werda',
'Wesselsbron',
'Willemsrust Silo',
'Winburg Silo',
'Wolmaransstad Silo',
'Wolwehoek',
'Mkondo',
'Panbult',
'Rietspruit Bunker',
'Ascent',
'Cornelia',
'Danielsrus',
'Frankfort',
'Jim Fouche',
'Lehau',
'Memel',
'Naboomspruit',
'Nutfield',
'Nylstroom',
'Petrus Steyn',
'Reitz',
'Roedtan',
'Settlers',
'Tweeling',
'Villiers',
'Vrede',
'Warden',
'Warmbad',
'Wilgerspruit Bunker',
'Windfield'
]

vv= ['AFGRI',
'BKB',
'GRAIN MANAGEMENT',
'KAAP AGRI',
'NWK',
'OVERBERG AGRI BEDRYWE',
'OOS-VRYSTAAT KOOPERASIE',
'PROCURO',
'SENTRAAL-SUID KOOPERASIE',
'SILOSTRAT',
'SENWES',
'TWK',
'VKB']

commodity = [
'0012GH',
'0024GH',
'0024GK',
'0024GL',
'0024GY',
'0070 ',
'0070GH',
'0070GK',
'0070GL',
'0070GX',
'0070GY',
'0070H',
'0071GH',
'0071GK',
'0071GL',
'0071GX',
'0072GH',
'0072GK',
'0072GL',
'0072GX',
'0072GY',
'0075GH',
'0075GK',
'0075GL',
'0075GX',
'0075GY',
'0076GH',
'0076GK',
'0076GL',
'0076GX',
'2811894WM1',
'2811894WM2',
'2811894WM3',
'2811894YM2',
'2811894YM3',
'AK',
'B1',
'B2',
'B3',
'b4',
'BSG',
'FH',
'FH1',
'GARSMGAND',
'GH1',
'GH2',
'GM IMP',
'GM1',
'H1',
'HWR',
'K ANDER',
'KA',
'KRG',
'SB',
'SB1',
'SBSOJA',
'SBSOJAOG',
'SP',
'UT',
'WM1',
'WM2',
'WM3',
'YM1',
'YM1 NON GMO',
'YM2',
'YM2 NON GMO',
'YM3',
'YM3 NON GMO',
'YMOG',
'VOERGARS',
'SOJA',
'SONNEBLOM',
'WIT MIELIES',
'GEEL MIELIES',
'KORING',
'MOUTGARS']


# Open the PDF
with pdfplumber.open("pdfs/7.pdf") as pdf:
    all_floats = []
    for page in pdf.pages:
        text = page.extract_text()
        text = text.replace(",","")

        sx = text.split('\n')
        dxc = ""
        for idx, ml in enumerate(sx):
            sxx = []
            floats = extract_floats(ml)
            if floats and len(floats) > 3:
                # print(" ".join(sx))
                for indx, i in enumerate(dd):
                      if i in " ".join(sx):
                            sxx.append(str(i))
                            sxx.append( cc[indx])
                for indx2, b in enumerate(jj):
                      if b.lower() in " ".join(sx).lower():
                            sxx.append(str(b))
                for indx3, c in enumerate(vv):
                      if c.lower() in " ".join(sx).lower():
                            sxx.append(str(c))
                # for indx4, com in enumerate(commodity):
                #       if com.lower() in " ".join(sx).lower():
                #             sxx.append(str(com))
                print(sxx + floats)
                desc = re.split(r'\d+', ml)
                for descc in desc:
                     if len(descc) > 4:
                        dxc = descc
                        print(dxc)
                print("Nexgro Company: " + sxx[-1] + " Co-Op Name: " + sxx[1] + " Folio Account number: " + sxx[0] + " Description: " + dxc 
                      + " Silo: " + sxx[2] + " Grade: " + (dxc.strip().split()[-1] if dxc.strip() else "") + " Units: " + floats[0] + " Price: " + floats[1] + " Vat: " + floats[2]  + " Total Incl: "  + floats[3])
            elif floats and len(floats) >= 3:
               # print(" ".join(sx))
                for indx, i in enumerate(dd):
                      if i in " ".join(sx):
                            sxx.append(str(i))
                            sxx.append( cc[indx])
                for indx2, b in enumerate(jj):
                      if b.lower() in " ".join(sx).lower():
                            sxx.append(str(b))
                for indx3, c in enumerate(vv):
                      if c.lower() in " ".join(sx).lower():
                            sxx.append(str(c))
                # for indx4, com in enumerate(commodity):
                #       if com.lower() in " ".join(sx).lower():
                #             sxx.append(str(com))
                print(sxx + floats)
                desc = re.split(r'\d+', ml)
                for descc in desc:
                     if len(descc) > 4:
                        dxc = descc
                        print(dxc)
                print("Nexgro Company: " + sxx[-1] + " Co-Op Name: " + sxx[1] + " Folio Account number: " + sxx[0] + " Description: " + dxc 
                      + " Silo: " + sxx[2] + " Grade: " + (dxc.strip().split()[-1] if dxc.strip() else "") + " Units: " + floats[0] + " Price: " + floats[1] + " Vat: " + floats[2]  + " Total Incl: "  + floats[2])

['B257201-G301', 'AGI', 'Malmesbury', 'BKB', '28.149', '22.95', '646.02']
 FEED BARLEY - GRADE 
Nexgro Company: BKB Co-Op Name: AGI Folio Account number: B257201-G301 Description:  FEED BARLEY - GRADE  Silo: Malmesbury Grade: GRADE Units: 28.149 Price: 22.95 Vat: 646.02 Total Incl: 646.02
['B257201-G301', 'AGI', 'Malmesbury', 'BKB', '646.02', '96.90', '742.92']
STORAGE CHARGE 
Nexgro Company: BKB Co-Op Name: AGI Folio Account number: B257201-G301 Description: STORAGE CHARGE  Silo: Malmesbury Grade: CHARGE Units: 646.02 Price: 96.90 Vat: 742.92 Total Incl: 742.92


In [78]:
data4

,claim_date,signup_date,signup_date2,claim_date2,check
0,1/9/19,1/1/19,2019-01-01,2019-01-09,False
1,1/10/19,1/5/19,2019-01-05,2019-01-10,False
2,1/9/20,1/5/19,2019-01-05,2020-01-09,False
3,1/8/21,1/5/19,2019-01-05,2021-01-08,False
4,1/8/22,1/5/19,2019-01-05,2022-01-08,False
...,...,...,...,...,...
49993,3/10/23,2/3/21,2021-02-03,2023-03-10,False
49994,3/17/21,2/3/21,2021-02-03,2021-03-17,False
49995,6/22/21,2/4/21,2021-02-04,2021-06-22,False
49996,6/30/22,2/4/21,2021-02-04,2022-06-30,False


In [79]:
import pandas as pd
data = pd.read_csv("/Users/sbusisophakathi/Downloads/row_health_wellness_program_analysis-main/data/customers.csv")
data2 = pd.read_csv("/Users/sbusisophakathi/Downloads/row_health_wellness_program_analysis-main/data/claims.csv")

data3 = data2.merge(data, on='customer_id', how='inner')
data4 = data3[['claim_date', 'signup_date']]



data4['signup_date2'] = pd.to_datetime(data4['signup_date'], format='%m/%d/%y').dt.date
data4['claim_date2'] = pd.to_datetime(data4['claim_date'], format='%m/%d/%y').dt.date


data4['check'] = data4['claim_date2'] == data4['signup_date2']
data4[data4['check'] == True]

/var/folders/9h/tb31qbc10y968kpjxnfyzzn40000gn/T/ipykernel_12563/2294399611.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data4['signup_date2'] = pd.to_datetime(data4['signup_date'], format='%m/%d/%y').dt.date
/var/folders/9h/tb31qbc10y968kpjxnfyzzn40000gn/T/ipykernel_12563/2294399611.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data4['claim_date2'] = pd.to_datetime(data4['claim_date'], format='%m/%d/%y').dt.date
/var/folders/9h/tb31qbc10y968kpjxnfyzzn40000gn/T/ipykernel_12563/2294399611.py:1

,claim_date,signup_date,signup_date2,claim_date2,check
175,2/15/21,2/15/21,2021-02-15,2021-02-15,True
232,2/17/21,2/17/21,2021-02-17,2021-02-17,True
377,2/27/21,2/27/21,2021-02-27,2021-02-27,True
2349,6/14/21,6/14/21,2021-06-14,2021-06-14,True
2777,7/7/21,7/7/21,2021-07-07,2021-07-07,True
...,...,...,...,...,...
47615,10/1/20,10/1/20,2020-10-01,2020-10-01,True
47870,10/13/20,10/13/20,2020-10-13,2020-10-13,True
48299,11/11/20,11/11/20,2020-11-11,2020-11-11,True
48463,11/18/20,11/18/20,2020-11-18,2020-11-18,True


In [4]:
import os
import base64
import pdfplumber
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow

# Define OAuth scopes
SCOPES = ["https://www.googleapis.com/auth/gmail.modify"]

# Save attachments here
SAVE_DIR = "attachments"
os.makedirs(SAVE_DIR, exist_ok=True)

# Authenticate Gmail API
def get_gmail_service():
    creds = None
    token_file = "token.json"
    creds_file = "creds.json"  # Replace with your credentials file

    if os.path.exists(token_file):
        creds = Credentials.from_authorized_user_file(token_file, SCOPES)

    if not creds or not creds.valid:
        flow = InstalledAppFlow.from_client_secrets_file(creds_file, SCOPES)
        creds = flow.run_local_server(port=0)
        with open(token_file, "w") as token:
            token.write(creds.to_json())

    return build("gmail", "v1", credentials=creds)

# Mark email as read
def mark_email_as_read(service, msg_id):
    service.users().messages().modify(
        userId="me",
        id=msg_id,
        body={"removeLabelIds": ["UNREAD"]}
    ).execute()
    print(f"✅ Marked email {msg_id} as read.")

# Fetch unread emails
def get_unread_emails():
    service = get_gmail_service()
    results = service.users().messages().list(userId="me", labelIds=["UNREAD"]).execute()
    messages = results.get("messages", [])

    if not messages:
        print("No unread emails found.")
        return

    for message in messages:
        msg_id = message["id"]
        msg = service.users().messages().get(userId="me", id=msg_id).execute()
        print(f"\n📩 Processing Email: {msg['snippet'][:50]}...")

        # Extract attachments
        extract_attachments(service, msg)

        # Mark email as read AFTER processing
        mark_email_as_read(service, msg_id)

# Extract attachments from email
def extract_attachments(service, msg):
    msg_id = msg["id"]
    for part in msg["payload"].get("parts", []):
        if part.get("filename") and "application/pdf" in part["mimeType"]:
            attachment_id = part["body"]["attachmentId"]
            attachment = service.users().messages().attachments().get(userId="me", messageId=msg_id, id=attachment_id).execute()
            
            file_data = base64.urlsafe_b64decode(attachment["data"])
            file_path = os.path.join(SAVE_DIR, part["filename"])

            with open(file_path, "wb") as f:
                f.write(file_data)

            print(f"📄 Saved PDF: {file_path}")
            extract_text_from_pdf(file_path)

# Extract text from PDF
def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = "\n".join(page.extract_text() for page in pdf.pages if page.extract_text())
        print(f"🔍 Extracted Text from {pdf_path}:\n{text[:500]}...\n")  # Print first 500 characters

# Run script
get_unread_emails()



📩 Processing Email: sdcsd...
📄 Saved PDF: attachments/AFGRI INV 9001811822 - UG.pdf
🔍 Extracted Text from attachments/AFGRI INV 9001811822 - UG.pdf:
COPY TAX CREDIT NOTE AFGRI Grain Silo Company (Pty) Ltd
Reg. Number: 2016/440533/07
VAT Reg No: 4130283874
Postal Address: PO Box 11054
Highveld Ext 73, Centurion
Physical Address: 12 Byls Bridge Boulevard
Centurion, 0046
Invoice No: 9001811822
Reference No: HT-2011-01013504
Document Date: 03 May 2024
UNIGRAIN (PTY) LTD
Po Box 488 Payment Details
Randfontein Bank name: ABSA
Account Name: AFGRI Grain Silo Company (Pty) Ltd
1760
Account No: 04095120081
South Africa Branch Code: 620503
VAT Reg No: ...



HttpError: <HttpError 403 when requesting https://gmail.googleapis.com/gmail/v1/users/me/messages/19541d3cf19ac54b/modify?alt=json returned "Request had insufficient authentication scopes.". Details: "[{'message': 'Insufficient Permission', 'domain': 'global', 'reason': 'insufficientPermissions'}]">

In [16]:
import os
import base64
import pdfplumber
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow

# Define OAuth scopes
SCOPES = ["https://www.googleapis.com/auth/gmail.modify"]

# Save attachments here
SAVE_DIR = "attachments"
os.makedirs(SAVE_DIR, exist_ok=True)

# Authenticate Gmail API
def get_gmail_service():
    creds = None
    token_file = "token.json"
    creds_file = "creds.json"  # Replace with your credentials file

    if os.path.exists(token_file):
        creds = Credentials.from_authorized_user_file(token_file, SCOPES)

    if not creds or not creds.valid:
        flow = InstalledAppFlow.from_client_secrets_file(creds_file, SCOPES)
        creds = flow.run_local_server(port=0)
        with open(token_file, "w") as token:
            token.write(creds.to_json())

    return build("gmail", "v1", credentials=creds)

# Mark email as read
def mark_email_as_read(service, msg_id):
    service.users().messages().modify(
        userId="me",
        id=msg_id,
        body={"removeLabelIds": ["UNREAD"]}
    ).execute()
    print(f"✅ Marked email {msg_id} as read.")

# Fetch unread emails
def get_unread_emails():
    service = get_gmail_service()
    results = service.users().messages().list(userId="me", labelIds=["UNREAD"]).execute()
    messages = results.get("messages", [])

    if not messages:
        print("No unread emails found.")
        return

    for message in messages:
        msg_id = message["id"]
        msg = service.users().messages().get(userId="me", id=msg_id).execute()
        # print(f"\n📩 Processing Email: {msg['snippet'][:50]}...")

        # Extract attachments
        extract_attachments(service, msg)

        # Mark email as read AFTER processing
        mark_email_as_read(service, msg_id)

# Extract attachments from email
def extract_attachments(service, msg):
    msg_id = msg["id"]
    for part in msg["payload"].get("parts", []):
        if part.get("filename") and "application/pdf" in part["mimeType"]:
            attachment_id = part["body"]["attachmentId"]
            attachment = service.users().messages().attachments().get(userId="me", messageId=msg_id, id=attachment_id).execute()
            
            file_data = base64.urlsafe_b64decode(attachment["data"])
            file_path = os.path.join(SAVE_DIR, part["filename"])

            with open(file_path, "wb") as f:
                f.write(file_data)

            print(f"📄 Saved PDF: {file_path}")
            extract_text_from_pdf(file_path)
            print('\n\n')

# Extract text from PDF
def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = "\n".join(page.extract_text() for page in pdf.pages if page.extract_text())
        print(f"🔍 Extracted Text from {pdf_path}:\n{text[:500000]}...\n")  # Print first 500 characters

# Run script
get_unread_emails()


📄 Saved PDF: attachments/SENWES INV - MySenwes_Invoice_9552018724 - UG.pdf
🔍 Extracted Text from attachments/SENWES INV - MySenwes_Invoice_9552018724 - UG.pdf:
Copy Tax Invoice
HEAD OFFICE Document No: 9552018724
PO BOX 31 Date : 08.05.2024
KLERKSDORP 2570
Registration number 1997/005336/06
TEL : (018) 464 7800
VAT Registration no 4950112195
FAKS: (018) 464 7771 Page 1 of 1
SENWES LIMITED
Account no. 400-401664 ZGG Plant No : G240
UNIGRAIN PTY LTD Plant Name: Raathsvlei Silo
PO Box 488 Ref Number : 4724251 - 4994121757
RANDFONTEIN Cust Order: G240A83939
1759 Contact : Melissa Wiese
Cont Tel : 018 4647893
Cont Email: melissa.wiese@senwes.co.za
Vat No : 4670140054
Item Description Quantity UoM Unit Price VAT Amount
_____________________________________________________________________________
000010 Verwyder Giftige Sade Soja 2-10%
6.560 TO 72.89 71.72 478.16
_____________________________________________________________________________
Subtotal 71.72 478.16
Total due (Currency : ZAR) 549.

In [ ]:
import os
import base64
import pdfplumber
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow

# Define OAuth scopes
SCOPES = ["https://www.googleapis.com/auth/gmail.modify"]

# Save attachments here
SAVE_DIR = "attachments"
os.makedirs(SAVE_DIR, exist_ok=True)

# Authenticate Gmail API
def get_gmail_service():
    creds = None
    token_file = "token.json"
    creds_file = "credentials.json"  # Replace with your credentials file

    if os.path.exists(token_file):
        creds = Credentials.from_authorized_user_file(token_file, SCOPES)

    if not creds or not creds.valid:
        flow = InstalledAppFlow.from_client_secrets_file(creds_file, SCOPES)
        creds = flow.run_local_server(port=0)
        with open(token_file, "w") as token:
            token.write(creds.to_json())

    return build("gmail", "v1", credentials=creds)

# Mark email as read
def mark_email_as_read(service, msg_id):
    service.users().messages().modify(
        userId="me",
        id=msg_id,
        body={"removeLabelIds": ["UNREAD"]}
    ).execute()

# Fetch unread emails
def get_unread_emails(service):
    results = service.users().messages().list(userId="me", labelIds=["UNREAD"]).execute()
    messages = results.get("messages", [])

    return messages

# Extract attachments from email
def extract_attachments(service, msg, save_dir):
    msg_id = msg["id"]
    for part in msg["payload"].get("parts", []):
        if part.get("filename") and "application/pdf" in part["mimeType"]:
            attachment_id = part["body"]["attachmentId"]
            attachment = service.users().messages().attachments().get(userId="me", messageId=msg_id, id=attachment_id).execute()
            
            file_data = base64.urlsafe_b64decode(attachment["data"])
            file_path = os.path.join(save_dir, part["filename"])

            with open(file_path, "wb") as f:
                f.write(file_data)

            return file_path

# Extract text from PDF
def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = "\n".join(page.extract_text() for page in pdf.pages if page.extract_text())
    return text


In [84]:
import unittest
from unittest.mock import patch, MagicMock
import os

class TestGmailScript(unittest.TestCase):
    
    @patch('googleapiclient.discovery.build')
    def test_get_gmail_service(self, mock_build):
        # Mock the returned service
        mock_service = MagicMock()
        mock_build.return_value = mock_service
        
        service = get_gmail_service()
        
        # Test if the build function was called
        mock_build.assert_called_with("gmail", "v1", credentials=None)
        self.assertEqual(service, mock_service)
    
    @patch('googleapiclient.discovery.build')
    def test_get_unread_emails(self, mock_build):
        # Mock Gmail API response
        mock_service = MagicMock()
        mock_build.return_value = mock_service
        
        # Mock the API call response
        mock_service.users().messages().list.return_value.execute.return_value = {
            "messages": [{"id": "msg1"}, {"id": "msg2"}]
        }
        
        # Call the function
        messages = get_unread_emails(mock_service)
        
        # Test if the API was called with correct parameters
        mock_service.users().messages().list.assert_called_with(userId="me", labelIds=["UNREAD"])
        self.assertEqual(len(messages), 2)
    
    @patch('googleapiclient.discovery.build')
    def test_extract_attachments(self, mock_build):
        # Mock Gmail API response for email parts
        mock_service = MagicMock()
        mock_build.return_value = mock_service
        mock_email = {
            "id": "msg1",
            "payload": {
                "parts": [
                    {"filename": "test.pdf", "mimeType": "application/pdf", "body": {"attachmentId": "att1"}}
                ]
            }
        }
        # Mock the attachment content
        attachment_data = {
            "data": base64.urlsafe_b64encode(b"test content").decode('utf-8')
        }
        mock_service.users().messages().attachments().get.return_value.execute.return_value = attachment_data
        
        # Test if the function downloads and saves the attachment
        file_path = extract_attachments(mock_service, mock_email, "attachments")
        self.assertTrue(os.path.exists(file_path))  # Check if file was saved
        os.remove(file_path)  # Clean up
    
    @patch('pdfplumber.open')
    def test_extract_text_from_pdf(self, mock_pdfplumber):
        # Mock PDF plumper to return fake text
        mock_pdf = MagicMock()
        mock_pdfplumber.return_value = mock_pdf
        mock_pdf.pages = [MagicMock(extract_text=MagicMock(return_value="Fake text"))]
        
        text = extract_text_from_pdf("test.pdf")
        
        self.assertIn("Fake text", text)


if __name__ == '__main__':
    unittest.main()


usage: ipykernel_launcher.py [-h] [-v] [-q] [--locals] [--durations N] [-f]
                             [-c] [-b] [-k TESTNAMEPATTERNS]
                             [tests ...]
ipykernel_launcher.py: error: argument -f/--failfast: ignored explicit argument '/Users/sbusisophakathi/Library/Jupyter/runtime/kernel-v3d94fb90c2059ea73745afde566d839c759d6572b.json'


AttributeError: 'tuple' object has no attribute 'tb_frame'